In [ ]:
# --- User inputs (interactive) ---
directory   = input("📁 Enter the folder path containing your CSVs: ").strip()
core_name   = input("🧪 Enter core name (e.g., LakeCore1): ").strip()
plot_title  = input("📊 Enter plot title: ").strip()
output_dir  = input("💾 Enter output directory (default '.'): ").strip() or "."
fig_w       = float(input("📏 Figure width (inches, default 2): ") or 2)
fig_h       = float(input("📐 Figure height (inches, default 10): ") or 10)

# Wentworth bins (µm) and labels
wentworth_bins   = [0, 4, 63, 2000]
wentworth_labels = ["Clay", "Silt", "Sand"]

figsize = (fig_w, fig_h)
print(f"\n✅ Ready! Core '{core_name}' — using data in: {directory}\n")


In [ ]:
#Cell 2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

plt.rcParams.update({"font.size": 15})


In [ ]:
#Cell 3
def extract_depths(file_stem: str):
    parts = file_stem.split("_")[-1].split("-")
    try:
        lo = float(parts[0].replace("mm", "").strip())
        hi = float(parts[1].replace("mm", "").strip())
        return lo, hi, (lo + hi) / 2
    except (ValueError, IndexError) as e:
        raise ValueError(
            f"Bad name '{file_stem}'. Use '..._<lower>-<upper>mm.csv'."
        ) from e


In [ ]:
#Cell 4
def read_multiple_data(directory: str) -> pd.DataFrame:
    frames = []
    for fn in os.listdir(directory):
        if not fn.endswith(".csv"): 
            continue
        fp = os.path.join(directory, fn)
        df = pd.read_csv(fp)
        lo, hi, mid = extract_depths(os.path.splitext(os.path.basename(fp))[0])
        df["Lower Depth"] = lo
        df["Upper Depth"] = hi
        df["Center Depth"] = mid
        frames.append(df)
    if not frames:
        raise FileNotFoundError(f"No CSVs in {directory}")
    return pd.concat(frames, ignore_index=True)


In [ ]:
def plot_wentworth_scale(data, bins, labels, title, figsize, out_pdf, gap_mm=20, colors=None):
    depths = np.sort(data["Center Depth"].unique())
    binned = pd.cut(data["um"], bins=bins, labels=labels, right=False, include_lowest=True, ordered=True)
    tmp = data.assign(class_=binned).groupby(["Center Depth", "class_"], observed=True)["Histogram (%) q3"].sum().reset_index()
    wide = tmp.pivot(index="Center Depth", columns="class_", values="Histogram (%) q3").reindex(labels, axis=1).fillna(0)
    cumu = wide.cumsum(axis=1).to_numpy(); fig, ax = plt.subplots(figsize=figsize)
    idx = np.arange(len(depths)); gaps = np.where(np.diff(depths) > gap_mm)[0]; segs = np.split(idx, gaps + 1)
    cols = colors or ["#1f77b4", "#17becf", "#bcbd22"]  # Clay, Silt, Sand
    for i, lab in enumerate(labels):
        left = np.zeros(len(depths)) if i == 0 else cumu[:, i-1]; right = cumu[:, i]
        for j, s in enumerate(segs):
            ys = depths[s]; yl = left[s]; yr = right[s]
            ypad = np.r_[ys[0]-1e-6, ys, ys[-1]+1e-6]
            ax.fill_betweenx(ypad, np.r_[yl[0], yl, yl[-1]], np.r_[yr[0], yr, yr[-1]],
                             color=cols[i], label=lab if j == 0 else None)
    ax.set(xlim=(0, 100), ylabel="Core Depth (mm)", xlabel="Grain Size (%)", title=title)
    ax.invert_yaxis(); ax.legend(loc="lower center", bbox_to_anchor=(0.5, -0.15), ncol=len(labels), frameon=False)
    fig.savefig(out_pdf, bbox_inches="tight", dpi=300); plt.close(fig); print(f"Plot saved as {out_pdf}")


In [ ]:
#Cell 6
combined_data = read_multiple_data(directory)
combined_data.head()


In [ ]:
plot_wentworth_scale(
    combined_data, wentworth_bins, wentworth_labels, plot_title, figsize, out_path,
    gap_mm=20, colors=["#653700", "#808080", "#E6DAA6"]  # Clay, Silt, Sand
)
